# Data preparation
Cleaning of the data
Splitting the data in training/testing set
Create datasets per repository
Doing anything else that needs to be done

In [2]:
# Auxiliary
import logging as log
import pymongo

import pandas as pd
from pandas import json_normalize

import textstat

name = './logs/03.log'
log.basicConfig(filename=name, level=log.DEBUG, force=True, format='%(asctime)s %(message)s: ', datefmt='%Y-%m-%d %H:%M:%S')

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']
coll_issues = db['01_Issues']

In [3]:
issues = coll_issues.find()
df = json_normalize(issues)

In [4]:
df

,_id,id,repo,sourceType,created,updated,description,duedate,labels,issuelinks,...,issuetype.subtask,project.id,watches.watchCount,status.statusCategory.name,project.projectCategory.id,originSprintId,sprintHistories,totalHistories,noOfIssues,plannedDuration
0,629209bb136340a01c2c6830,Jira467709,Jira,indirect,2015-09-24T05:24:14.000+0000,2019-08-26T05:10:48.000+0000,If user clones via Hosted Repositories in Sour...,None,[],"[{'id': '194138', 'self': 'https://jira.atlass...",...,False,14510,4,NaN,10031,Jira2660,3,30,3.0,0.0
1,629209bc136340a01c2c6832,Jira526575,Jira,indirect,2015-11-17T11:20:56.000+0000,2020-07-01T05:00:31.000+0000,I just got an update to 2.0.5.6 (which curious...,None,[],"[{'id': '202604', 'self': 'https://jira.atlass...",...,False,12910,21,NaN,10031,Jira2743,3,48,3.0,10.0
2,629209bc136340a01c2c6833,Jira518198,Jira,indirect,2015-11-11T09:06:25.000+0000,2020-07-01T05:00:33.000+0000,"I have put ;s5|7ZY""S1/>86Xn""73N83lv=s&-V]7v as...",None,[],"[{'id': '419600', 'self': 'https://jira.atlass...",...,False,12910,3,NaN,10031,Jira2743,3,48,3.0,10.0
3,629209bc136340a01c2c6834,Jira473042,Jira,indirect,2015-10-08T06:14:49.000+0000,2020-07-01T05:00:36.000+0000,Note: This affects 2.0.5.3 as well. (It is not...,None,[],[],...,False,12910,2,NaN,10031,Jira2660,3,30,3.0,0.0
4,629209bc136340a01c2c6835,Jira472411,Jira,indirect,2015-10-06T19:35:48.000+0000,2020-07-01T05:00:38.000+0000,After installing/upgrading to Bitbucket server...,None,[],"[{'id': '196313', 'self': 'https://jira.atlass...",...,False,12910,5,NaN,10031,Jira2660,3,30,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155437,62921406136340a01c2eea7d,Apache13069994,Apache,indirect,2017-05-08T12:59:29.000+0000,2018-01-25T16:29:55.000+0000,ARIA could benefit from having global configur...,None,[],[],...,False,12320626,2,NaN,10220,Apache336,19,95,6.0,15.0
155438,62921406136340a01c2eea7e,Apache13049213,Apache,indirect,2017-03-08T09:42:52.000+0000,2018-02-15T12:57:35.000+0000,Using a plugin currently requires a user first...,None,"[plugins, wishlist]","[{'id': '12503404', 'self': 'https://issues.ap...",...,False,12320626,2,NaN,10220,Apache336,19,95,6.0,15.0
155439,62921406136340a01c2eea7f,Apache13037378,Apache,indirect,2017-01-24T15:26:56.000+0000,2018-03-19T09:28:37.000+0000,Our exceptions are currently far too general.\...,None,[],[],...,False,12320626,2,NaN,10220,Apache336,19,95,6.0,15.0
155440,62921406136340a01c2eea81,Apache13019966,Apache,indirect,2016-11-10T22:17:42.000+0000,2018-01-07T18:18:40.000+0000,The parser code currently makes heavy usage of...,None,[],"[{'id': '12516440', 'self': 'https://issues.ap...",...,False,12320626,2,NaN,10220,Apache329,1,16,1.0,15.0


In [18]:
for key in df.keys():
    print(key)

_id
id
repo
sourceType
created
updated
description
duedate
labels
issuelinks
subtasks
comments
resolution.name
resolution.description
priority.id
assignee.key
status.name
status.description
votes.votes
issuetype.name
issuetype.subtask
project.id
watches.watchCount
status.statusCategory.name
project.projectCategory.id
originSprintId
sprintHistories
totalHistories
noOfIssues
plannedDuration


In [9]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)


In [12]:
missing_value_df[missing_value_df['percent_missing']>0]

,column_name,percent_missing
originSprintId,originSprintId,0.052109
noOfIssues,noOfIssues,0.059829
plannedDuration,plannedDuration,0.908377
comments,comments,2.647290
priority.id,priority.id,2.948367
resolution.description,resolution.description,5.046255
resolution.name,resolution.name,5.046255
assignee.key,assignee.key,6.311679
project.projectCategory.id,project.projectCategory.id,15.670797
description,description,16.603621


originSprintId = remove, because they also do not have any sprint data
noOfIssues = replace with median
plannedDuration = replace with median
comments = replace with median
priority.id =
resolution.description = 
resolution.name = 
assignee.key = random out of possible values: using any statistical method might lead to a heavier bias than random
project.projectCategory.id = 
description = replace textstat equiv column with median
duedate = drop, too many missing
status.statusCategory.name = rerun 

In [13]:
# investigate priority.id
df['priority.id'].unique()

array(['3', '1', '4', '2', nan, '10003', '10001', '10000', '10004', '5',
       '7', '10002', '10300', '10200', '6', '10102', '10101', '10103'],
      dtype=object)

In [ ]:
# sprintHistories

# text complexity

In [24]:
def stringify(input):
    return str(input)

In [25]:
# new readability and complexity levels
df['description'] = df['description'].apply(stringify)
df['description.flesch'] = df['description'].apply(textstat.flesch_reading_ease)
df['description.mcalpine_eflaw'] = df['description'].apply(textstat.mcalpine_eflaw)